### Предобработка данных

Перед тем как обучать модель, нужно подготовить для этого данные. 

Современные нейонные сети, и в особенности генеративные модели, обучаются на очень больших данных. Обработка их с помощью всем известной библиотеки `pandas` затруднительна по ряду причин (нет многопточности, кеширования, неудобный формат хранения, ...)

Для работы с большими объемами структурированных данных существует несколько библиотек (`arrow`, `pil`, ...). В этом ноутбуке мы будем использовать библиотеку `datasets` от *hugging-face*, основанную на `arrow`. 

In [ ]:
import pandas as pd

from pathlib import Path
from datasets import load_dataset

In [ ]:
data = load_dataset('csv', 
    data_files={
        'train': ... 
    })

In [ ]:
data

In [ ]:
# train_test_split
data = ...  # YOUR CODE HERE

In [ ]:
data['train'][0:2]  # First two rows of the dataset of type Dict[FeatureName, List[values]]

Обработка данных осуществляется с помощью 2х основных функций -- `filter` и `map` . 

In [29]:
? data.filter

Signature:
 data.filter(
    function,
    with_indices=False,
    input_columns: Union[str, List[str], NoneType] = None,
    batch_size: Union[int, NoneType] = 1000,
    remove_columns: Union[List[str], NoneType] = None,
    keep_in_memory: bool = False,
    load_from_cache_file: bool = True,
    cache_file_names: Union[Dict[str, Union[str, NoneType]], NoneType] = None,
    writer_batch_size: Union[int, NoneType] = 1000,
    fn_kwargs: Union[dict, NoneType] = None,
    num_proc: Union[int, NoneType] = None,
) -> 'DatasetDict'
Docstring:
Apply a filter function to all the elements in the table in batches
and update the table so that the dataset only includes examples according to the filter function.
The transformation is applied to all the datasets of the dataset dictionary.

Args:
    function (`callable`): with one of the following signature:
        - `function(example: Dict) -> bool` if `with_indices=False`
        - `function(example: Dict, indices: int) -> bool` if `with_indices

In [ ]:
? data.map

Оставим только те примеры, где context_1 не пустой

In [ ]:
data = ...  # YOUR CODE HERE

Приводим каждый пример из датасета к виду одной строки

In [ ]:
from typing import Dict

FIRST_SPEAKER_TOKEN = '@@ПЕРВЫЙ@@'
SECOND_SPEAKER_TOKEN = '@@ВТОРОЙ@@'

CONTEXT_COLS = ['context_3', 'context_2', 'context_1']
RESPONSE_COL = ['response']

def convert_to_dialog(example: Dict[str, str]) -> Dict[str, str]:
    """
        Convert example row to dialogs str format
    """
    # YOUR CODE HERE
    ...

assert convert_to_dialog(
    {
        'context_3': 'привет',
        'context_2': 'привет!',
        'context_1': 'как дела?',
        'response': 'супер)'
    }
) == {'text': '@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет! @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@ супер)'}
assert convert_to_dialog(
    {
        'context_1': 'как дела?',
        'response': 'супер)'
    }
) == {'text': '@@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@ супер)'}

In [ ]:
data = ...  # YOUR CODE HERE

Теперь датасет необходимо токенизировать, точно также как мы это делали при знакомстве с хаггинфейс

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')

In [ ]:
from typing import List


def tokenize_example(example: Dict[str, str]):
    # YOUR CODE HERE
    ...

In [ ]:
data = ...  # YOUR CODE HERE

### Обучение

Создаем модель, которую будем обучать, а также вспомогательные классы -- trainer, training_args и datacollator

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForCausalLM.from_pretrained('tinkoff-ai/ruDialoGPT-medium').to(device)

Указываем основные параметры обучения и создаем класс Trainer (https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer)

In [ ]:
arguments = {
    'output_dir': './training_output',  # path to save the model's checkpoints
    'per_device_train_batch_size': 16,  # batch size per GPU/CPU for training
    'gradient_accumulation_steps': 4,  # number of batches to accumulate gradient
    'max_steps': 500,  # total number of optimizer.step() calls
    'save_steps': 100,  # save every save_steps
    'eval_steps': 100,  # run evaluation every eval_steps
    'dataloader_num_workers': 0,  # number of workers for data loading (default: 0)
    'save_total_limit': 2,  # total number of checkpoints to save, delete older checkpoints when reached
}

trainer = ... # YOUR CODE HERE

Запускаем обучение

In [ ]:
# YOUR CODE HERE

#### Пробуем обученный чекпоинт

In [ ]:
from transformers import AutoModelWithLMHead

checkpoint_path = 'path/to/your/checkpoint-100'

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelWithLMHead.from_pretrained(checkpoint_path)